# Verification Circuit Synthesis

Example notebook for verification circuit synthesis task.

In [ ]:
import sys
sys.path.append(r'../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

## 01 - Simplest Example

In the simplest example, you only need to give the encoding circuit. It supports `stim.Circuit` or `qiskit.QuantumCircuit` object

By default, the gate set will be [CX, CZ, H] and use all-to-all qubit connectivity.

In [ ]:
import stim
from rlftqc.verification_circuit_synthesis import VerificationCircuitSynthesis

## Define the encoding circuit
## For example, circuit that prepares zero logical of 7 qubit Steane code.
circ = stim.Circuit("""
H 0 1 3
CX 0 6 1 5 0 4 3 4 3 5 5 6 0 2 1 2
""")

## Create class
vcs = VerificationCircuitSynthesis(circ, ignore_z_errors = True)


In [ ]:
## Train the agent
vcs.train()

In [ ]:
## Run the agent and get the prepared circuit
vcs.run()

In [ ]:
## One can also curstomize the folder name to save the circuits
vcs.run(results_folder_name='results')

In [ ]:
## Log the result if needed
vcs.log()

In [ ]:
## One can also customize the folder name to save log
vcs.log(results_folder_name='logs')

## 02 - Advanced Example 

This part shows how to customize the training configuration.

In [ ]:
from rlftqc.logical_state_preparation import LogicalStatePreparation

## Define the target stabilizers
## For example, zero logical of 7 qubit Steane code.
target = ["ZZZZZZZ",
        "ZIZIZIZ",
        "XIXIXIX",
        "IZZIIZZ",
        "IXXIIXX",
        "IIIZZZZ",
        "IIIXXXX",
        ]

## Create class
lsp = LogicalStatePreparation(target)


Change the number of possible gates for training with the max_steps.

In [ ]:
lsp = LogicalStatePreparation(target, max_steps = 100)

Change seed for training.

In [ ]:
lsp = LogicalStatePreparation(target, seed = 123)

For more advanced training configurations, we can change the training config

In [ ]:
lsp.training_config

In [ ]:
# NUM_AGENTS change the number of parallel agents to train (default: 1).
lsp.training_config['NUM_AGENTS'] = 5

# TOTAL_TIMESTEPS change the number of total timesteps for training (default: 5e5), increase this for longer training.
lsp.training_config['TOTAL_TIMESTEPS'] = 1e7
